In [19]:
!pip install pyranges
!pip install pandas
!pip install numpy
!pip install ray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 44.7 MB/s eta 0:00:000m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.8/530.8 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 14.2 MB/s eta 0:00:00


In [137]:
import pyranges as pr
import pandas as pd
import numpy as np
import argparse
import warnings

from stats_voting import *
from io_.schemas import *
from io_.fileops import load_rdc, load_BED, make_pyranges_format, save_file_custom

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None


In [138]:
rdc = '~/gpfs/test_outdir/merge/0h_1_merged.tab'
annot = '/gpfs/anikolskaya/rnachrom_pipeline_faster/test_files/test_annot/All_RNAs_mM_DB_pipe_new.0-corrected_annot.tab'
outdir = '~/gpfs/test_outdir'
rna_parts = True
no_ribo = False
no_stat = False

In [139]:
def run_annotation_and_voting(rdc, annot, outdir, rna_parts, no_ribo, no_stat):
    """
    Annotate RNA-parts of contacts, apply a voting procedure and remove
    contacts derived from ribosomal DNA if specified.
    
    """

    d = {}
    save_names = {'singletons':       singleton_suffix,
                  'selected_annot':   selected_ann_suffix,
                  'voted':            voted_suffix, 
                  'complement_annot': complement_ann_suffix,
                  'voted_noribo':     voted_nr_suffix
                  }
    empty = []
    
    gene_ann = load_BED(annot).rename(columns={'name': 'gene_name', 'source': 'Source'})
    gene_ann = make_pyranges_format(gene_ann, strand = True, save_old_names = False)

    gene_ann.gene_length = gene_ann.lengths()
    gene_ann = gene_ann[['gene_name', 'gene_type', 'Source', 'gene_length']]
    
    if rna_parts:
        coln_rdc = rdc_BED_sample[:5] + [rdc_BED_sample[-1]]
        coln_voted = voted_BED[:5] + voted_BED[9:13]
    else:
        coln_rdc = rdc_BED_sample.copy()
        coln_voted = voted_BED

    cnts = load_rdc(rdc, header = 0, names = coln_rdc, dtype = rdc_dtypes)
    
    cnts.drop(cnts.filter(regex='_cigar').columns, axis=1, inplace=True)
    print(cnts)
    cnts, old_cols = make_pyranges_format(cnts, strand = True)

    d['selected_annot'], d['complement_annot'] = annotate_rdc(cnts, gene_ann, cpus = 2)
    del gene_ann

    d = dict(map(lambda item: (item[0], item[1].drop(like="annot$").as_df()), d.items()))
    save_file_custom(d['selected_annot'].iloc[:,:len(coln_voted)], outdir, rdc, save_names['selected_annot'], hdr=coln_voted)
    d['voted'] = vote(d['selected_annot'])
    del d['selected_annot']

    cnts = cnts.as_df()
    
    cnts.colnames = old_cols

    d['singletons'] = cnts[~cnts['id'].isin(d['voted']['id'])]
    
    if no_ribo:
        d['voted_noribo'] = remove_ribo(d['voted'])

    for name, dfr in d.items():
        if not dfr.empty:
            if name == 'singletons':
                save_file_custom(dfr, outdir, rdc, save_names[name], hdr=coln_voted[:-3])
            else:
                save_file_custom(dfr.iloc[:,:len(coln_voted)], outdir, rdc, save_names[name], hdr=coln_voted)
                    
        else:
            save_file_custom(dfr, outdir, rdc, save_names[name], hdr=coln_voted)
            empty.append(name)

    for empty_df in empty:
        d.pop(empty_df, None)

    if not no_stat:
        calculate_stats(d, rdc, outdir)

    return 

In [140]:
def annotate_rdc(contacts: pr.PyRanges, annot: pr.PyRanges, cpus: int) -> pr.PyRanges:
    """
    Create annotated genomic intervals from a RNA-DNA contacts file.

    Parameters
    ----------
    contacts: PyRanges
        RDC-like file converted to PyRanges
        
    annot: PyRanges
        A GTF- or BED-like file converted to PyRanges

    Returns
    -------
    PyRanges: 
         1. Intervals annotated by genes on selected strand (selected_annot)
         2. Intervals annotated by genes on the complementing strand (complement_annot)

    """
    
    print(contacts.df)
    print(annot.df)
    all_annot = contacts.join(annot, how = 'left', strandedness = False, suffix = '_annot', nb_cpu = cpus)
    print(all_annot.df)
    strand_match = all_annot.Strand == all_annot.Strand_annot
    gene_found = all_annot.gene_name != '-1'
    
    return all_annot[(strand_match & gene_found)], all_annot[(~strand_match & gene_found)]


In [141]:

def vote(an_contacts: pd.DataFrame) -> pd.DataFrame:
    """
    Apply voting procedure to RNA parts of contacts. In the case of unambiguous annotation 
    the preference is given to the gene with more dense coverage of RNA-parts.

    """
    an_contacts['count'] = an_contacts['gene_name'].map(an_contacts['gene_name'].value_counts())
    an_contacts['density'] = (an_contacts['count'] / 
                                 an_contacts['gene_length']) * 100000

    if any('gencode' in s for s in an_contacts['Source'].unique()):
        an_contacts['gencode'] = np.where(an_contacts['Source'].str.contains('gencode'), 'gencode', 'other')
        return an_contacts.sort_values(['density', 'gencode'], ascending=(False, True)).drop_duplicates('id', keep='first').drop('gencode', axis=1)
    
    else:
        return an_contacts.sort_values('density', ascending=False).drop_duplicates('id', keep='first')



def remove_ribo(contacts: pd.DataFrame, ribo_list = ['rRNA', 'rRNA_pseudogene', 'rRNA_RepM']) -> pd.DataFrame:
    """
    Removes contacts mapping to ribosomal DNA 
    
    """
    return contacts[~contacts.gene_type.isin(ribo_list)]


In [142]:

run_annotation_and_voting(rdc, annot, outdir, rna_parts, no_ribo, no_stat)


     rna_chr    rna_bgn    rna_end               id rna_strand srr_id
0       chr4  104731117  104731138  SRR5035944.1284         60      -
1      chr17   39848576   39848596   SRR5035944.261         60      -
2       chr9   45823704   45823725   SRR5035944.894         60      +
3       chrX   59471475   59471496  SRR5035944.1391         60      +
4       chr8  105204180  105204200   SRR5035944.281         60      -
...      ...        ...        ...              ...        ...    ...
2127    chr4  141489624  141489644   SRR5035946.403         60      +
2128    chr4  134244455  134244475   SRR5035946.309         60      +
2129   chr10   61099525   61099546   SRR5035946.255         60      +
2130    chr4  132310347  132310368   SRR5035946.665         60      -
2131   chr16   93878747   93878768   SRR5035946.869         60      -

[2132 rows x 6 columns]
Empty DataFrame
Columns: []
Index: []
      Chromosome      Start        End      gene_name Strand  gene_type  \
0           chr1  1206

2024-01-17 01:30:00,510	INFO worker.py:1724 -- Started a local Ray instance.


Empty DataFrame
Columns: []
Index: []


AttributeError: ('PyRanges object has no attribute', 'Strand')